# Part 2: TODO

## Zonal Statistics

## Masking

## Reprojection

# The Grand Finale

# Appendix

## Supplemental Materials
- Zarr documentation:
- Xarray documentation:
- Rioxarray documentation:
- Icechunk documentation:
- VirtualiZarr documentation:
- Dask documentation:
- Arraylake documentation:

## On Chunking

Data chunking in Xarray (with Dask) is a way to break up large datasets into smaller, manageable pieces ("chunks") that can be processed lazily and in parallel. It’s essential when working with out-of-core data — data too big to fit into memory.

But how do we know what chunks to set? While we could rely on `chunks="auto"` to determine optimal chunks for us, let's walk through an example with some actual math (I know, scary! 😱)

### Example

Take the following example `DataArray` of size `(y: 40,000, x: 40,000)` with `dtype=float32`:

- The array size is `(y: 40,000, x: 40,000)`, so there are `40,000 x 40,000 = 1.6 billion values` with `dtype=float32`
- Each `float32` is 4 bytes, so the whole array is `1.6e9 x 4 bytes = 6.4 GB`

Let's say we want to keep **chunk size between ~50MB to 200MB** for efficiency and to optimize for **access patterns** (i.e. processing entire rows vs entire tiles)

Let's get even more specific and target ~100 MB chunks. Each `float32` value is 4 bytes, so:
```
chunk_size = (chunk_y, chunk_x)
chunk_memory = chunk_y * chunk_x * 4 bytes
```

#### **Option 1: Chunk by tiles (e.g. 1000 x 1000)**
```
chunks = {"y": 1000, "x": 1000}
memory_per_chunk = 1000 * 1000 * 4 = 4 MB
```
🚫 Too small — leads to `40,000 / 1,000 = 400 chunks` per axis = **160,000 chunks total** 😱 (overhead!)

#### **Option 2: Bigger tiles (e.g. 4000 x 4000)**
```
chunks = {"y": 4000, "x": 4000}
memory_per_chunk = 4000 * 4000 * 4 = 64 MB
```
✅ This results in 10 `y` chunks and 10 `x` chunks, so `10 x 10 = 100 total chunks`. This strikes a nice balance between chunk size and number.

#### **Option 3: Slightly larger tiles (e.g. 5000 x 5000)**
```
chunks = {"y": 5000, "x": 5000}
memory_per_chunk = 5000 * 5000 * 4 = 100 MB
```
✅ This hits the target chunk size exactly. The array would be divided into `40,000 / 5,000 = 8` chunks along each axis, so `8 x 8 = 64 chunks total`

#### **Option 4: Row-wise chunks (e.g. full row per chunk)**
```
chunks = {"y": 1, "x": 40000}
memory_per_chunk = 1 * 40000 * 4 = 160 KB
```
⚠️ These chunks are way too small — 40,000 tiny chunks, one per row! This might make sense only if you only ever access full rows, but still leads to a huge number of chunks and high Dask scheduler overhead.

#### **Option 5: Chunk by wide horizontal strips (e.g. 5000 x 40000)**
```
chunks = {"y": 5000, "x": 40000}
memory_per_chunk = 5000 * 40000 * 4 = 800 MB
```
🚫 Too big — although it reduces the number of chunks to 8, each chunk is 800 MB, which is likely too large for efficient Dask scheduling and memory usage unless you're on a beefy machine.

#### **Recommendation:**
Stick with something like Option 3: 5000 x 5000

→ Hits ~100MB sweet spot

→ Balanced chunk count (64)

→ Good for general-purpose tile-based access and parallelism

If you know your access pattern is row-oriented, you might bias chunks more like `{"y": 10000, "x": 4000}` or `{"y": 4000, "x": 10000}` depending on whether you need fewer chunks along y or x, but aim to stay between 50MB–200MB per chunk.